<a href="https://colab.research.google.com/github/Eshapriyadarshi/Crowd_estimation/blob/main/Crowd_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Documentation:
Object counting and crowd estimation is computer vision problem which is widely applicable in surevelliance. However state of the art models  of instance segmentation like Mask-RCNN fail to provide us the accuracy when it coes to crowd estimation. Here is an excellent paper whose work i tried to replicate.https://openaccess.thecvf.com/content_CVPR_2019/papers/Cholakkal_Object_Counting_and_Instance_Segmentation_With_Image-Level_Supervision_CVPR_2019_paper.pdf

This paper has outperformed the state of the art model of image-level supervised instance segmentation like Peak response mapping.
This approach is built upon an ImageNet pre-trained network backbone (ResNet50). The proposed network architecture
has two output branches: image classification and density
branch. The image classification branch estimates the presence or absence of objects, whereas the density branch predicts the global object count and the spatial distribution of object instances by constructing a density map. We remove the global pooling layer from the backbone and adapt the fully connected layer with a 1 × 1 convolution having 2P channels as output. We divide these 2P channels equally between the image classification and density branches. We then add a 1×1 convolution having C output channels in each branch, resulting in a fully convolutional network.
 The resnet50 backbone is trained with peak response mapping with enabled peak stimulation on PASCAL VOC and COCO dataset.

 the efficiency of the approach can be seen here:
 https://nbviewer.jupyter.org/github/GuoleiSun/CountSeg/blob/master/demo/images/demo-coco.ipynb

 this entire approach uses pytorch I could not succeed completely to implement the same in Tensorflow and keras as the peak response mapping model is only available in pytorch and cant even converted to a tensorflow framework using ONNX because of the complex structure of the model.

 link to the github of the implementation:
 https://github.com/GuoleiSun/CountSeg

In [1]:
#Installing the Coco dataset api
!pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-npda9xg8
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-npda9xg8
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266458 sha256=ea1cd29a52bf0bbb223a12b83f2817b0cd0f08beb57d9d3a52237ce164afbbbf
  Stored in directory: /tmp/pip-ephem-wheel-cache-iqp8bemc/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.2
    Uninstalling pycocotools-2.0.2:
      Successfully uninstalled pycocotools-2.0.2


Importing Necessary modules

In [27]:
!pip install Keras-Applications
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop, Adam
from PIL import Image
from numpy import asarray
from keras.applications.resnet50 import ResNet50
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input
from keras.layers import Input
from keras.models import model_from_json
from pycocotools.coco import COCO

     |████████████████████████████████| 51kB 2.9MB/s 


 ImageNet pre-trained network backbone (ResNet50)

In [ ]:
#remove the top level global pooling layer
Backbone = ResNet50(weights=None,include_top=False)

# summarize input and output shape
print('Inputs: %s' % Backbone.inputs)
print('Outputs: %s' % Backbone.outputs)

Inputs: [<tf.Tensor 'input_5:0' shape=(None, None, None, 3) dtype=float32>]
Outputs: [<tf.Tensor 'conv5_block3_out/Relu_4:0' shape=(None, None, None, 2048) dtype=float32>]


In [ ]:
Backbone.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________